In [11]:
from pennylane import numpy as np
import pennylane as qml 

In [71]:
N = 5
p = 3


dev = qml.device("default.qubit", wires = N )


In [72]:
w = np.arange(N)
def circuit(param, # p X (N+N) thetas and phis  
            w #wires list 
            ):
    (p, _)  = param.shape
    N = len(w)
    # prepare |+> state with Hadamard gate 
    for i in range(N):
        qml.Hadamard(wires = w[i])
    # parametrized circuit with depth p 
    for step in range(p):
        for i in range(N):
            qml.RX(param[step, i], wires = w[i]) # exp(-i X phi/2)
        for i in range(N-1):
            qml.CNOT(wires = w[i: i+2])
            qml.RZ(param[step, i], wires = w[i+1])
            qml.CNOT(wires = w[i: i+2])
        qml.CNOT(wires = [w[N-1], w[0]])
        qml.RZ(param[step, N-1], wires = w[N-1])
        qml.CNOT(wires = [w[N-1], w[0]])


# Hamiltonian definition H = sum_i (- Z_i * Z_{i+1})
coeffs = -1 * np.ones((N,))
obs = [qml.PauliZ(i)@qml.PauliZ(i+1) for i in range(N-1)]
obs.append(qml.PauliZ(N-1)@qml.PauliZ(0))
H = qml.Hamiltonian(coeffs, obs)
print(H)


  (-1.0) [Z0 Z1]
+ (-1.0) [Z1 Z2]
+ (-1.0) [Z2 Z3]
+ (-1.0) [Z3 Z4]
+ (-1.0) [Z4 Z0]


In [73]:

@qml.qnode(dev, interface='autograd')
def cost_fn(param):
    circuit(param, w = range(N))
    return qml.expval(H)

opt = qml.GradientDescentOptimizer(stepsize = 0.1 )


In [74]:
param = np.random.randn(p, 2*N, requires_grad = True)
energy = [cost_fn(param)]
max_iterations = 300 
conv_tol = 1e-06 
for n in range(max_iterations):
    param, prev_energy = opt.step_and_cost(cost_fn, param)
    energy.append(cost_fn(param))
    conv = np.abs(energy[-1]-prev_energy)

    if n%20 ==0: 
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")


Step = 0,  Energy = -1.20268371 Ha
Step = 20,  Energy = -2.33608782 Ha
Step = 40,  Energy = -2.78327061 Ha
Step = 60,  Energy = -3.04597973 Ha
Step = 80,  Energy = -4.37143747 Ha
Step = 100,  Energy = -4.99489650 Ha
Step = 120,  Energy = -4.99791026 Ha
Step = 140,  Energy = -4.99900645 Ha
Step = 160,  Energy = -4.99950305 Ha
Step = 180,  Energy = -4.99974509 Ha
Step = 200,  Energy = -4.99986755 Ha
Step = 220,  Energy = -4.99993071 Ha
Step = 240,  Energy = -4.99996363 Ha

Final value of the ground-state energy = -4.99997001 Ha


In [62]:
m = qml.matrix(H)
np.linalg.eig(m)

(array([-4.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  4.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  4.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -4.+0.j]),
 array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.